In [ ]:
import utility_module_2_v_3 as uu
import numpy as np
import matplotlib.pyplot as plt
import numpy.ma as ma
from numpy import meshgrid

from geopy.geocoders import Nominatim
from tqdm import tqdm
import indices_no_leap as ii

# Figure 3

In [ ]:
map_bounds=[65, 40, 100, 5]

## Import data

In [ ]:
output_figname = ''
path_out_IMD = '/home/vikram/BCSD_data/India_P_BCSDd'+output_figname

In [ ]:
def getclosest_ij(lats,lons,latpt,lonpt):
    dis_sq1=(lats-latpt)**2
    dis_sq2=(lons-lonpt)**2
    
    minidex_lat=dis_sq1.argmin()
    minidex_lon=dis_sq2.argmin()
    
    return minidex_lon,minidex_lat

def extract_regions(data_input,lat_obs,lon_obs):
    
    data_to_indices = []
    for address in ['Kolkata','Mumbai','New Delhi', 'Chennai']:
        geolocator = Nominatim(user_agent="Your_Name")
        location = geolocator.geocode(address)
        [ind_lon,ind_lat] = getclosest_ij(lat_obs.data,lon_obs.data,location.latitude,location.longitude)

        data_to_indices.append(data_input[:, ind_lat,ind_lon])


    regional_boxes = [[72,29,77,25],[77,24,84,18],[87,30,97,21]]

    for co_ords in regional_boxes:

        lat_index_123 = np.logical_and(lat_obs >= co_ords[3], lat_obs <= co_ords[1])
        lon_index_123 = np.logical_and(lon_obs >= co_ords[0], lon_obs <= co_ords[2])

        data_to_indices.append(np.nanmean(data_input[:, lat_index_123,:][:,:,lon_index_123],axis=(1,2)))

    data_to_indices.append(np.nanmean(data_input,axis=(1,2)))
    
    # print(np.array(data_to_indices).shape)
    data_indices = {}
    for index_name in ["P_ann","IAV","R20mm","Rx1day", "CDD","R99p"]:
        data_indices[index_name] =[]   
    for itera, data_temp in enumerate(data_to_indices):
        
        for index_name,func in zip(["P_ann","IAV","R20mm","Rx1day", "CDD","R99p"],
                                   [ii.sum_annualy, ii.sum_annualy, ii.R20mm_annualy,ii.Rx1day_annualy,ii.CDD_annualy,ii.R99p_annualy]):
            # print(itera,data_temp.shape,data_temp.shape[0]//365)
            if index_name == "IAV":
                # print("Var loop entered.")
                data_indices[index_name].append(np.nanvar(func(data_temp,data_temp.shape[0]//365)))

            else:
                # print("NN loop entered.")

                data_indices[index_name].append(np.nanmean(func(data_temp,data_temp.shape[0]//365)))
        
    # print("Inxde",np.array(data_indices).shape,np.array(data_indices[index_name]).shape) 
    return data_indices

In [ ]:
gcm_list = ['GFDL_CM3','CanESM2','MIROC_ESM','MRI_ESM1','ACCESS1_0','NorESM1_M','CM5A_LR','CNRM_CM5','MPI_ESM_MR']
index_list = ['CDD','Rx1day','R20mm','R99p']

gcm_meta=np.load(path_out_IMD + '/bcsd_imd_metadata_'+gcm_list[0]+'.npy',allow_pickle=True)
Nyears_train = gcm_meta.item()['Nyears_train']
N_future_start = 2045-2006
st=gcm_meta.item()['st']
en=gcm_meta.item()['en']

lat_obs=gcm_meta.item()['lat_obs']
lat_gcm_01=gcm_meta.item()['lat_gcm']
lon_obs=gcm_meta.item()['lon_obs']
lon_gcm_01=gcm_meta.item()['lon_gcm']
mask=gcm_meta.item()['mask']

N1=lat_obs.shape[0]
N2=lon_obs.shape[0]


data_all_final_observations =[]

imd_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/data_obs_imd_pred.npy'))[365*Nyears_train:]
data_all_final_observations.append(extract_regions(imd_test,lat_obs,lon_obs))

data_all_final_gcms =[]
data_all_final_gcms_future =[]
data_all_final_bcs_gcms =[]
data_all_final_bcs_gcms_future =[]
data_all_final_random_fields =[]

for gcm_name in tqdm(gcm_list):
    
    gcm_meta=np.load(path_out_IMD + '/bcsd_imd_metadata_'+gcm_name+'.npy',allow_pickle=True)
    lat_gcm_01=gcm_meta.item()['lat_gcm']
    lon_gcm_01=gcm_meta.item()['lon_gcm']

    gcm_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/data_gcm_'+gcm_name+'_imd_pred.npy'))[365*Nyears_train:]
    data_gf_test=np.zeros(imd_test.shape)
    for i in range(gcm_test.shape[0]):
        data_gf_test[i,:,:]=uu.regrid(gcm_test[i,:,:],lat_gcm_01,lon_gcm_01,lat_obs,lon_obs)
    data_all_final_gcms.append(extract_regions(data_gf_test,lat_obs,lon_obs))
    
    gcm_future=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/data_gcm_'+gcm_name+'_future.npy'))[365*N_future_start:]
    data_gf_future=np.zeros([gcm_future.shape[0],imd_test.shape[1],imd_test.shape[2]])
    for i in range(data_gf_future.shape[0]):
        data_gf_future[i,:,:]=uu.regrid(gcm_future[i,:,:],lat_gcm_01,lon_gcm_01,lat_obs,lon_obs)
    data_all_final_gcms_future.append(extract_regions(data_gf_future,lat_obs,lon_obs))
    
    out_id='pre_'+gcm_name 
    bcsd_gcm_test=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/BCSD_outputs_'+out_id+'.npy'))[365*Nyears_train:]
    data_all_final_bcs_gcms.append(extract_regions(bcsd_gcm_test,lat_obs,lon_obs))

    bcsd_gcm_future=uu.correct_nan_inf_neg(np.load(path_out_IMD + '/future_BCSD_outputs_'+out_id+'.npy'))[365*N_future_start:]
    data_all_final_bcs_gcms_future.append(extract_regions(bcsd_gcm_future,lat_obs,lon_obs))
    
    
for j in tqdm(range(100)):
    out_id='pre_rnd_new_'+str(j)  
    bcsd_rnd_test=uu.correct_nan_inf_neg((np.load(path_out_IMD + '/BCSD_outputs_'+out_id+'.npy')))[365*Nyears_train:]
    data_all_final_random_fields.append(extract_regions(bcsd_rnd_test,lat_obs,lon_obs))



In [ ]:
plt.figure(figsize = (10,4))

N_esm =len(gcm_list)
N_random = 10



index=  'R99p'
units = "mm"

ylims = [[0,250],[0,250],[0,250],[0,320],
         [0,100],[0,180],[0,250],[0,100]]



index=  'IAV'
units = "x 10000 mm2"


ylims = [[0,150000],[0,300000],[0,100000],[0,450000],
         [0,50000],[0,150000],[0,250000],[0,50000]]


index=  'P_ann'
units = "mm"

ylims = [[0,2500],[0,3000],[0,2000],[0,3000],
         [0,1000],[0,1800],[0,3000],[0,2000]]



index=  'CDD'
units = "# days"
    
ylims = [[0,100],[0,250],[0,250],[0,320],
         [0,250],[0,180],[0,75],[0,50]]
    
    
index=  'Rx1day'
units = "mm"
    
ylims = [[0,150],[0,250],[0,150],[0,250],
         [0,100],[0,100],[0,80],[0,40]]

index=  'R20mm'
units = "# days"
    
ylims = [[0,40],[0,50],[0,20],[0,50],
         [0,20],[0,25],[0,60],[0,3]]
    
    
markers=['o','^','s','p','P','*','h','+','x','X','D']

data_to_indices_label  =  ['Kolkata', 'Mumbai', 'New Delhi' , 'Chennai' , 
                           'Western India', 'Central India' , 'NE India','All India']

# plt.plot( 0,0, 'D',markersize = 10,color ='w',linestyle = 'None',mfc='none',label ='Random Fields')
for pliter in range(8):
    plt.subplot(2,4,pliter +1)
    plt.plot(1,data_all_final_observations[0][index][pliter], marker = markers[0],markersize = 10,linestyle = 'None')

    for gi in range(N_esm):
        plt.plot( 2 , data_all_final_gcms[gi][index][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , data_all_final_bcs_gcms[gi][index][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

    plt.plot(list(4 for x in range(N_random)) ,list(data_all_final_random_fields[x][index][pliter] for x in range(N_random)),
         markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
    
    plt.xticks(range(1,5),["Obs","ESM","BCSD","Ran"])
    plt.title(data_to_indices_label[pliter])
    
    plt.xlim([0.5,4.5])
    plt.ylim(ylims[pliter])
    if index == "IAV":
        plt.yticks(np.linspace(ylims[pliter][0],ylims[pliter][1],6),np.linspace(ylims[pliter][0]/10000,ylims[pliter][1]/10000,6))
    else:
        # plt.yticks(np.linspace(ylims[pliter][0],ylims[pliter][1],6),np.linspace(ylims[pliter][0]/10000,ylims[pliter][1]/10000,6))
        pass

    plt.ylabel(units)
    
plt.tight_layout()
# plt.suptitle("Comparison of " + index + " for bias-corrected results")
plt.suptitle( index )
plt.savefig("./qq_future/" + index + ".png", dpi =150, facecolor = 'w')







In [ ]:
plt.figure(figsize = (10,4))
N_esm =len(gcm_list)
N_random = 10



markers=['o','^','s','p','P','*','h','+','x','X','D']


data_to_indices_label  =  ['Kolkata', 'Mumbai', 'New Delhi' , 'Chennai' , 
                           'Western India', 'Central India' , 'NE India','All India']

index=  'IAV'
units = "x 10000 mm2"


ylims = [[0,150000],[0,300000],[0,100000],[0,450000],
         [0,50000],[0,150000],[0,250000],[0,50000]]


index=  'P_ann'
units = "mm"

ylims = [[0,2500],[0,3000],[0,2000],[0,3000],
         [0,1000],[0,1800],[0,3000],[0,2000]]


index=  'R99p'
units = "mm"
ylims = [[0,300],[0,300],[0,250],[0,350],
         [0,80],[0,180],[0,250],[0,100]]

index=  'CDD'
units = "# days"
ylims = [[0,120],[0,250],[0,250],[0,320],
         [0,250],[0,180],[0,75],[0,50]]
    
index=  'Rx1day'
units = "mm"   
ylims = [[0,150],[0,250],[0,150],[0,250],
         [0,100],[0,100],[0,80],[0,40]]

index=  'R20mm'
units = "# days"  
ylims = [[0,40],[0,50],[0,20],[0,50],
         [0,15],[0,25],[0,60],[0,3]]


# plt.plot( 0,0, 'D',markersize = 10,color ='w',linestyle = 'None',mfc='none',label ='Random Fields')
for pliter in range(8):
    plt.subplot(2,4,pliter +1)
    plt.plot(1,data_all_final_observations[0][index][pliter], marker = markers[0],markersize = 10,linestyle = 'None')

    for gi in range(N_esm):

        plt.plot( 2 , data_all_final_gcms_future[gi][index][pliter],
             markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , data_all_final_bcs_gcms_future[gi][index][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

    plt.plot(list(4 for x in range(N_random)) ,list(data_all_final_random_fields[x][index][pliter] for x in range(N_random)),
         markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
    
    
    plt.xticks(range(1,5),["Obs","ESM","BCSD","Ran"])
    plt.title(data_to_indices_label[pliter])
    
    plt.xlim([0.5,4.5])
    plt.ylim(ylims[pliter])
    plt.ylabel(units)
    if index == "IAV":
        plt.yticks(np.linspace(ylims[pliter][0],ylims[pliter][1],6),np.linspace(ylims[pliter][0]/10000,ylims[pliter][1]/10000,6))
    else:
        # plt.yticks(np.linspace(ylims[pliter][0],ylims[pliter][1],6),np.linspace(ylims[pliter][0]/10000,ylims[pliter][1]/10000,6))
        pass
plt.tight_layout()
# plt.suptitle("Comparison of " + index + " for bias-corrected results")
plt.suptitle( "Future : " + index )
plt.savefig("./qq_future/future_" + index + ".png", dpi =150, facecolor = 'w')

# Others

In [ ]:
plt.figure(figsize = (10,4))
N_esm =len(gcm_list)
N_random = 10
index=  'R99p'
units = 'mm'

markers=['o','^','s','p','P','*','h','+','x','X','D']


data_to_indices_label  =  ['Kolkata', 'Mumbai', 'New Delhi' , 'Chennai' , 
                           'Western India', 'Central India' , 'NE India','All India']

# plt.plot( 0,0, 'D',markersize = 10,color ='w',linestyle = 'None',mfc='none',label ='Random Fields')
for pliter in range(8):
    plt.subplot(2,4,pliter +1)
    plt.plot(1,data_all_final_observations[0][index][pliter], marker = markers[0],markersize = 10,linestyle = 'None')

    for gi in range(N_esm):
        plt.plot( 2 , data_all_final_gcms[gi][index][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 3 , data_all_final_bcs_gcms[gi][index][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 4 , data_all_final_gcms_future[gi][index][pliter],
             markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])

        plt.plot( 5 , data_all_final_bcs_gcms_future[gi][index][pliter],
                 markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])


    plt.plot(list(6 for x in range(N_random)) ,list(data_all_final_random_fields[x][index][pliter] for x in range(N_random)),
         markers[gi],markersize = 10,linestyle = 'None',mfc='none',label = gcm_list[gi])
    
    plt.xticks(range(1,7),["Obs","ESM","BC","ESM","BC","Ra"],rotation = 90)
    plt.title(data_to_indices_label[pliter])
    plt.xlim([0,7])
    if pliter < 4:
        plt.ylim([0,200])
    elif pliter <7:
        plt.ylim([0,100])
    else:
        plt.ylim([0,30])
plt.tight_layout()

In [ ]:
prints(imd_train,imd_test,data_gf_train,data_gf_test,bcsd_gcm_train,bcsd_gcm_test,bcsd_rnd_train,bcsd_rnd_test,data_gf_future)

## Checking imported data

In [ ]:

data_train = [imd_train,data_gf_train,data_gf_future,bcsd_gcm_future]
# data_test = [imd_test,data_gf_test,data_gf_future,bcsd_gcm_future]


plt.figure(figsize=(10,10))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.pcolor(data_train[i][250,:,:])
  plt.colorbar()

plt.figure(figsize=(20,10))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.plot(data_train[i][:,7,7])
    
plt.figure(figsize=(10,10))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.pcolor(data_test[i][250,:,:])
  plt.colorbar()

plt.figure(figsize=(20,10))
for i in range(4):
  plt.subplot(4,4,i+1)
  plt.plot(data_test[i][:,7,7])



In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import geopandas as gpd
from geopy.geocoders import Nominatim



lats,lons = (np.arange(0,40,1),np.arange(65,100,1))
XX12,YY12 = np.meshgrid(lons,lats)

fig, ax = plt.subplots()

for address in ['Kolkata','Mumbai','New Delhi', 'Chennai']:
    geolocator = Nominatim(user_agent="Your_Name")
    location = geolocator.geocode(address)
    [ind_lon,ind_lat] = getclosest_ij(lats,lons,location.latitude,location.longitude)
    plt.plot(lons[ind_lon],lats[ind_lat],marker='*',markersize= 25)

ind =gpd.read_file('/home/vikram/BCSD_Shapefiles/ind.shp')
ind.plot(ax=ax,facecolor="none")

regional_boxes = [[72,29,77,25],[77,24,84,18],[87,30,97,21]]
for co_ords in regional_boxes:
    ax.add_patch(Polygon([(co_ords[0],co_ords[1]),
                           (co_ords[0],co_ords[3]),
                           (co_ords[2],co_ords[3]),
                           (co_ords[2],co_ords[1])],facecolor='none',edgecolor= 'k'))

plt.show()




In [ ]:
from geopy.geocoders import Nominatim

def getclosest_ij(lats,lons,latpt,lonpt):
    dis_sq1=(lats-latpt)**2
    dis_sq2=(lons-lonpt)**2
    
    minidex_lat=dis_sq1.argmin()
    minidex_lon=dis_sq2.argmin()
    
    return minidex_lon,minidex_lat

func = ii.R20mm_annualy




data_to_indices_spatial = [imd_train,data_gf_test,bcsd_gcm_test,data_gf_future,bcsd_gcm_future]
data_indices_all = []
for data_input in data_to_indices_spatial:
    data_to_indices = []
    for address in ['Kolkata','Mumbai','New Delhi', 'Chennai']:
        geolocator = Nominatim(user_agent="Your_Name")
        location = geolocator.geocode(address)
        [ind_lon,ind_lat] = getclosest_ij(lat_obs.data,lon_obs.data,location.latitude,location.longitude)

        data_to_indices.append(data_input[:, ind_lat,ind_lon])


    regional_boxes = [[72,29,77,25],[77,24,84,18],[87,30,97,21]]

    for co_ords in regional_boxes:

        lat_index_123 = np.logical_and(lat_obs >= co_ords[3], lat_obs <= co_ords[1])
        lon_index_123 = np.logical_and(lon_obs >= co_ords[0], lon_obs <= co_ords[2])

        data_to_indices.append(np.nanmean(data_input[:, lat_index_123,:][:,:,lon_index_123],axis=(1,2)))

    data_to_indices.append(np.nanmean(data_input,axis=(1,2)))
    
    
    data_to_indices_label  =  ['Kolkata', 'Mumbai', 'New Delhi' , 'Chennai' , 'Western India', 'Central India' , 'NE India']
    data_indices = {}
    for data_temp in data_to_indices:
        for index_name in ["R20mm","Rx1day", "CDD","R99p"]:
            data_indices[index_name] =[]   
        
        for index_name,func in zip(["R20mm","Rx1day", "CDD","R99p"],
                                   [ii.R20mm_annualy,ii.Rx1day_annualy,ii.CDD_annualy,ii.R99p_annualy]):
        
            data_indices[index_name].append(np.nanmean(func(data_temp,data_temp.shape[0]//365)))
        
    data_indices_all.append(data_indices)

In [ ]:
data_labels = ['imd_train','data_gf_test','bcsd_gcm_test','data_gf_future','bcsd_gcm_future']

for i in range(5):
    plt.plot(np.array(data_indices_all)[i],marker ='*',linestyle = 'none',label = data_labels[i] )
plt.legend()